In [1]:
import pandas as pd

In [5]:
series_result = pd.read_csv('../data/geo-annotation/res.series-disease-all-platforms.1.csv', index_col=0).set_index('series')
series_result[:10]

,doid,disease,samples_count,platforms
series,,,,
GSE3,DOID:4467,renal clear cell carcinoma,342,"['GPL9', 'GPL10']"
GSE16,DOID:1612,breast cancer,25,['GPL28']
GSE17,DOID:440,neuromuscular disease,2,['GPL4']
GSE31,DOID:161,keratosis,5,['GPL4']
GSE49,DOID:1612,breast cancer,10,['GPL179']
GSE53,DOID:1612,breast cancer,26,['GPL170']
GSE59,DOID:1612,breast cancer,68,"['GPL167', 'GPL169']"
GSE62,DOID:2237,hepatitis,6,"['GPL167', 'GPL168', 'GPL169']"
GSE88,DOID:11054,urinary bladder cancer,31,['GPL80']


In [6]:
import pymongo
db = pymongo.MongoClient().scraper_meta

In [24]:
all_series = pd.DataFrame(list(db.series.find({'organism': 'Homo sapiens'}, {'_id':0, 'accession': 1, 'platforms': 1, 'samples': 1})))\
               .assign(samples_count=lambda d: d['samples'].map(len))\
               .drop('samples', axis=1) \
               .set_index('accession')
all_series.head()    

,platforms,samples_count
accession,,
GSE43950,[GPL10379],14
GSE7547,[GPL5104],189
GSE7563,[GPL4882],12
GSE7606,[GPL887],123
GSE43860,[GPL6604],56


In [26]:
all_series_with_result = all_series.join(series_result, rsuffix='_all').drop('platforms_all', axis=1).drop('samples_count_all', axis=1)
all_series_with_result.head()

,platforms,samples_count,doid,disease
accession,,,,
GSE43950,[GPL10379],14,NaN,NaN
GSE7547,[GPL5104],189,DOID:3393,coronary artery disease
GSE7563,[GPL4882],12,NaN,NaN
GSE7606,[GPL887],123,NaN,NaN
GSE43860,[GPL6604],56,NaN,NaN


In [30]:
bucket = []
for index, row in all_series_with_result.iterrows():
    if not row['platforms']:
        bucket.append(row)
    else:    
        for platform in row['platforms']:
            rowc = row.copy()
            rowc['platforms'] = platform
            bucket.append(rowc)
all_series_with_result_expanded = pd.DataFrame(bucket)
all_series_with_result_expanded.head()

,platforms,samples_count,doid,disease
GSE43950,GPL10379,14,NaN,NaN
GSE7547,GPL5104,189,DOID:3393,coronary artery disease
GSE7563,GPL4882,12,NaN,NaN
GSE7606,GPL887,123,NaN,NaN
GSE43860,GPL6604,56,NaN,NaN


In [32]:
(all_series_with_result_expanded[all_series_with_result_expanded.doid.notnull()].shape,
    all_series_with_result_expanded.shape
)

((10445, 4), (23548, 4))

In [37]:
d = all_series_with_result_expanded.reset_index()
d.head()

,index,platforms,samples_count,doid,disease
0,GSE43950,GPL10379,14,NaN,NaN
1,GSE7547,GPL5104,189,DOID:3393,coronary artery disease
2,GSE7563,GPL4882,12,NaN,NaN
3,GSE7606,GPL887,123,NaN,NaN
4,GSE43860,GPL6604,56,NaN,NaN


In [76]:
t = pd.DataFrame()
t['not_null'] = d[d.doid.notnull()].groupby('platforms').samples_count.sum()
t['all'] = d.groupby('platforms').samples_count.sum()
print(t.sum())
t.sort('not_null', ascending=False)[:20]

not_null    529020
all         984863
dtype: int64


,not_null,all
platforms,,
GPL570,74557,132700
GPL96,31711,51962
GPL10558,15571,40619
GPL6244,14431,29955
GPL6801,13094,20397
GPL8490,12397,23437
GPL3718,10753,19107
GPL13534,10632,30306
GPL3720,10219,15608


In [49]:
import sys
sys.path.append('..')

In [50]:
from importlib import reload
from os.path import exists

import networkx as nx
import numpy as np
import pandas as pd
import lib.obo
import lib.classification.elastic
import lib.classification.validation
import lib.utils
import data
import pymonad

reload(lib.classification.validation)
reload(lib.classification.elastic)
reload(lib.obo)
reload(lib.utils)
reload(data)

from lib.classification.elastic import annotate_index, collapse_matches, build_synonyms_graph
from lib.utils import map_series, expand_dataframe, expand, flatten
from lib.classification.validation import correct_ratio, incorrect_ratio, perfect_match
from geo_annotation.import_ontology_es import import_ontology
from elasticsearch import Elasticsearch
from lib.obo import read_ontology, Ontology
pd.set_option('display.width', 512)

In [51]:
ontology = read_ontology('../data/geo-annotation/doid-patched.obo',
                         exclude_duplicates=True)

Read obo graph
Name: 
Type: DiGraph
Number of nodes: 6595
Number of edges: 6594
Average in degree:   0.9998
Average out degree:   0.9998


In [73]:
root_node = 'DOID:4'
# root_node = 'DOID:162'
def descendants_level(graph, node, level):
    if level == 0:
        return [node]
    else:
        successors = graph.successors(node)
        res = []
        for succ in successors:
            res += descendants_level(graph, succ, level - 1)
        return res    
higer_nodes = [n for n in descendants_level(ontology.graph, root_node, 2)]
len(higer_nodes)

131

In [74]:
import numpy as np
def find_node(doid):
    return [node for node in higer_nodes if ontology.has_path(node, doid)][0]

d_top = d.assign(doid_higher=lambda d: d.doid.map(lambda doid: np.nan if pd.isnull(doid) else find_node(doid)))
# d_top[:10]

In [67]:
(
    d_top.dropna()
    .groupby('doid_higher')
    .samples_count
    .sum()
    .to_frame()
    .reset_index()
    .assign(disease=lambda d: d['doid_higher'].map(lambda doid: ontology.meta[doid].name))
    .sort('samples_count', ascending=False)
)


,doid_higher,samples_count,disease
14,DOID:162,417307,cancer
7,DOID:1287,14079,cardiovascular system disease
12,DOID:1579,11165,respiratory system disease
18,DOID:2914,10648,immune system disease
1,DOID:0060037,9858,developmental disorder of mental health
22,DOID:77,9402,gastrointestinal system disease
23,DOID:863,9164,nervous system disease
5,DOID:0060158,9093,acquired metabolic disease
15,DOID:17,8497,musculoskeletal system disease
10,DOID:1561,6643,cognitive disorder


In [82]:
# root_node = 'DOID:4'
def generalize_diseases(root_disease, level):
    root_node = root_disease
    def descendants_level(graph, node, level):
        if level == 0:
            return [node]
        else:
            successors = graph.successors(node)
            res = []
            for succ in successors:
                res += descendants_level(graph, succ, level - 1)
            return res    
    higer_nodes = [n for n in descendants_level(ontology.graph, root_node, level)]
    len(higer_nodes)

    import numpy as np
    def find_node(doid):
        res= [node for node in higer_nodes if ontology.has_path(node, doid)]
        return np.nan if not res else res[0]


    d_top_cancer = d.assign(doid_higher=lambda d: d.doid.map(lambda doid: np.nan if pd.isnull(doid) else find_node(doid)))
    d_top_cancer[:10]

    return (
        d_top_cancer.dropna()
        .groupby(['doid_higher'])
    #     .groupby(['doid_higher', 'platforms'])
        .samples_count
        .sum()
        .to_frame()
        .reset_index()
        .assign(disease=lambda d: d['doid_higher'].map(lambda doid: ontology.meta[doid].name))
        .sort('samples_count', ascending=False)
    )


In [100]:
generalize_diseases('DOID:162', 1)

,doid_higher,samples_count,disease
0,DOID:0050686,378010,organ system cancer
1,DOID:0050687,39297,cell type cancer


In [89]:
d.doid.unique().shape

(604,)

In [ ]:
# root_node = 'DOID:4'
root_node = 'DOID:162'
def descendants_level(graph, node, level):
    if level == 0:
        return [node]
    else:
        successors = graph.successors(node)
        res = []
        for succ in successors:
            res += descendants_level(graph, succ, level - 1)
        return res    
higer_nodes = [n for n in descendants_level(ontology.graph, root_node, 2)]
len(higer_nodes)

import numpy as np
def find_node(doid):
    res= [node for node in higer_nodes if ontology.has_path(node, doid)]
    return np.nan if not res else res[0]
        

d_top_cancer = d.assign(doid_higher=lambda d: d.doid.map(lambda doid: np.nan if pd.isnull(doid) else find_node(doid)))
d_top_cancer[:10]

(
    d_top_cancer.dropna()
    .groupby(['doid_higher'])
#     .groupby(['doid_higher', 'platforms'])
    .samples_count
    .sum()
    .to_frame()
    .reset_index()
    .assign(disease=lambda d: d['doid_higher'].map(lambda doid: ontology.meta[doid].name))
    .sort('samples_count', ascending=False)
)
